# Google's Data Analytics Capstone Project - Track 1

## Introduction

In this project, we'll act as a junior data analyst working on the marketing analyst team at Bellabeat, a high-tech manufacturer of health-focused
products for women. Where Urška Sršen, cofounder and Chief Creative Officer of Bellabeat, believes that analyzing smart
device fitness data could help unlock new growth opportunities for the company. We have been asked to analyze smart device data to gain insights into how customers are using their devices and help guide the marketing strategy for Bellabeat.

The main tools we'll use for this analysis will be SQL, R and Tableau.

## Ask

We'll begin by defining our business task:
    `Identify trends in smart device usage and use these trends to influence Bellabeat marketing strategy`
Which we can then break down into the following questions:
* What are some trends in smart device usage?
* How could these trends apply to Bellabeat customers?
* How could these trends help influence Bellabeat marketing strategy?
## Prepare

We are referred to the following <a href="https://www.kaggle.com/datasets/arashnic/fitbit" target="_blank">Fitbit fitness tracker</a> dataset (Public domain, available through Kaggle) which contains data from fitbit users, including information for physical activity, heart rate, and sleep monitoring.
This information is divided in 18 CSV documents which contain the information at different levels such as hourly or by minute.

We begin exploring these documents in BigQuery and after some exploratory analysis we encounter certain limitations with this dataset.

![Total users in daily activity](img/users_daily_activity.png) ![Total users in sleep activity](img/users_sleep_activity.png)

* Data includes 33 users but doesn't disclose gender nor age
* Not all segments (sleep, activity, weight) have the same number of observations
* Sample size too small to make valid assumptions


